## parsing harps website

* [HARPS bank](http://www.mpia.de/homes/trifonov/HARPS_RVBank.html)

In [82]:
from urllib.request import urlopen, urlretrieve
from os.path import exists
from os.path import join
import sys

import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_3d
from astropy.io import ascii


BASE = 'http://www.mpia.de/homes/trifonov/'
URL = BASE+'HARPS_RVBank.html'

def get_harps_database(clobber=False):
    fp = 'harps_db.csv'
    if not exists(fp) or clobber:    
        #scrape html table
        table = pd.read_html(url, header=0)
        #choose first table
        df = table[0]
    #     #define column names
    #     df.columns = df.loc[0].values
    #     #remove duplicate row
    #     df = df.drop(0,axis=0)

        #coordinates
        coords = SkyCoord(ra=df['RA'], dec=df['DEC'], unit=(u.hourangle,u.deg))
        df['RA_deg'] = coords.ra.deg
        df['DEC_deg'] = coords.dec.deg

        #save
        df.to_csv(fp,index=False)
        print('Saved: {}'.format(fp))
    else:
        df = pd.read_csv(fp)
        
    return df

In [2]:
df = get_harps_database(clobber=False)

In [128]:
def query_target(target, df, dist=1, unit=u.arcsec, verbose=False):
    coords = SkyCoord(ra=df['RA_deg'], dec=df['DEC_deg'], unit=u.deg)
    
    idxs = target.separation(coords)<dist*unit
    #search distance
    if idxs.sum() > 0:
        res = df[idxs]
        if verbose:
            print(df.loc[idxs,df.columns[7:14]])
        return res
    
    else:
        idx, sep2d, dist3d = match_coordinates_3d(target, coords, nthneighbor=1)
        nearest_obj = df.iloc[[idx]]['Target'].values[0]
        print('Nearest HARPS obj to target is {} (d={}\")'.format(nearest_obj,sep2d.arcsec[0]))
        return None

In [129]:
ra=279.272#+2/60
dec=5.292#+2/60

target = SkyCoord(ra=ra, dec=dec, unit=u.deg)
res = query_target(target, df, dist=1)
res

Nearest HARPS obj to target is BD+053829 (d=1.0632846949151857")


In [5]:
df.columns

Index(['Target', 'RA', 'DEC', 'Sp. Type', 'T_eff [K]', 'V [mag]', 'G [mag]',
       'Data product plots', 'Pre-upgrade DRS', 'Post-upgrade DRS',
       'Pre-upgrade standard SERVAL', 'Post-upgrade standard SERVAL',
       'Pre-upgrade mlc SERVAL (use these)',
       'Post-upgrade mlc SERVAL(use these)', 'RA_deg', 'DEC_deg'],
      dtype='object')

In [71]:
all_data_products_names = df.columns[7:14].values
all_data_products_names

array(['Data product plots', 'Pre-upgrade DRS', 'Post-upgrade DRS',
       'Pre-upgrade standard SERVAL', 'Post-upgrade standard SERVAL',
       'Pre-upgrade mlc SERVAL (use these)',
       'Post-upgrade mlc SERVAL(use these)'], dtype=object)

In [72]:
def get_rv(res, col='Pre-upgrade DRS', return_fp=True):
    assert col in all_data_products_names
    if col=='Data product plots':
        return NotImplementedError
    else:
        folder = res['Target'].values[0]+'_RVs'
        filename = res[col].values[0]
        rv = ascii.read(join(BASE,folder,filename)).to_pandas()
    #     rv.columns = 'BJD RV RV_err'.split()
        if return_fp:
            return rv,filename
        else:
            return rv

In [ ]:
rv = get_rv(res)
rv.head()

In [ ]:
%matplotlib inline
from astroplan.plots import plot_finder_image

ax,hdu=plot_finder_image(target,reticle=True,survey='DSS2 Red')
ax.scatter(target.ra.deg, target.dec.deg, transform=ax.get_transform('fk5'), s=300,
           edgecolor='red', facecolor='none')

In [1]:
import sys
sys.path.append('..')
import archive_digger as ad

In [2]:
ad.save_tics(outdir='../data')

Saved: ../data/TOIs.csv
Saved: ./tics_from_alerts.txt
